In [12]:
!git clone https://github.com/GiancarloSaran/MCHMC.git

Cloning into 'MCHMC'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 35 (delta 12), reused 19 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 320.64 KiB | 3.17 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [21]:
import sys
import os

notebook_dir=os.getcwd()

sys.path.append(os.path.join(notebook_dir, 'MCHMC'))

In [22]:
import torch
import numpy as np
import sys

from utils import checkpoint, warning
import utils
import MCHMC, MCLMC
import autotuning as aut
import integration_schemes as integ
import functions as funct
import visualization as vis

In [ ]:
debug = True

# ALGORITHM PARAMETERS
d = 1000  # dimension of the problem
N = 10000 # number of steps

# AUTOTUNING PARAMETERS
L_init = 10
N_prerun_eps = 100
iterations_eps = 10
N_prerun_L = 100

In [ ]:
eps, X, E = aut.tune_eps(d, L_init, funct.standard_cauchy, N_prerun_eps, iterations=iterations_eps, debug=debug)
epsilon_opt = eps[-1]

sigma_eff = aut.s_eff(X)
L_opt = aut.tune_L(d, sigma_eff, epsilon_opt, N_prerun_L, funct.standard_cauchy, debug)

K = int(L_opt//epsilon_opt) #  steps between bounces

checkpoint(f"Number of steps N = {N}")
checkpoint(f"Steps between bounces K = {K}")
checkpoint(f"Number of bounces: {int(N//K)}")

In [ ]:

X, E, ESS, B_squared = MCHMC.MCHMC_bounces(d, N, L_opt, epsilon_opt, funct.standard_cauchy, integ.leapfrog, debug=debug)

#plotting the results
vis.plot_cauchy(X)